## Exercise 17.1
***Purpose***: Apply linear model to quadratic data and do a
posterior predictive check.
### A
Change the script `Jags-Ymet-XmetSsubj-MrobustHier-Example.R` so it uses
the family income data (`file="Guber1999data.csv"`). (Notice how simple it is to do that.)

* Are the data well described by the linear model?
* What exactly is the “systematic discrepancy” between model and data? Don’t just say, “the data are curved.” What exactly does that mean in terms of how the data deviate from the model and where?


### B
Is there a way of rejecting the linear model without reference to any othermodel?
In other words, how might you compute a “Bayesian p value” for this situation? Should
you? See Kruschke (2013b) for information and references.

## Exercise 17.2
***Purpose***: Observe the autocorrelation in JAGS when data are
not standardized.
Change the (simple, nonhierarchical) JAGS program `Jags-Ymet-Xmet-Mrobust.R` so that it uses the raw data instead of the standardized data. Be sure
to rename it so that you don’t destroy the original program.

You will have to remove
the standardization from the data block and remove the transformation to the original
scale. Make sure that the prior is appropriate for the original scale (perhaps use the
prior in the Stan version for guidance). Show the diagnostic graphs of the chains and
discuss. Finally, run the chains a long time, with thinning if needed to conserve computer
memory, and show that the chains eventually converge to the same posterior as when
using standardized data.


## Exercise 17.3
***Purpose***: Examining chain convergence in JAGS and Stan.

![Figure 17.5](img/fig_17.5.png)

As mentioned in Footnote 5 (p. 504), both JAGS and Stan show some difficulty converging
when the quadratic-trend model is applied to the fictitious data of Figure 17.5. The
JAGS programs are provided in files `Jags-Ymet-XmetSsubj-MrobustHierQuadWt-
Example.R` and `Jags-Ymet-XmetSsubj-MrobustHierQuadWt.R`. The corresponding
Stan programs are provided in files `Stan-Ymet-XmetSsubj-MrobustHierQuadWt-
Example.R` and `Stan-Ymet-XmetSsubj-Mrobust HierQuadWt.R`. The Stan model
specification is shown below so that you can study it and compare it to the JAGS version
in Section 17.4 without having to be at your computer:

```
data {
int<lower=1> Nsubj ;
int<lower=1> Ntotal ;
real y[Ntotal] ;
real x[Ntotal] ;
real<lower=0> w[Ntotal] ;
int<lower=1> s[Ntotal] ;
}
transformed data {
// Standardize the data:
real zx[Ntotal] ;
real zy[Ntotal] ;
real zw[Ntotal] ;
real wm ;
real xm ;
real ym ;
real xsd ;
real ysd ;
xm <- mean(x) ;
ym <- mean(y) ;
wm <- mean(w) ;
xsd <- sd(x) ;
ysd <- sd(y) ;
for ( i in 1:Ntotal ) { // could be vectorized...?
zx[i] <- ( x[i] - xm ) / xsd ;
zy[i] <- ( y[i] - ym ) / ysd ;
zw[i] <- w[i] / wm ;
}
}
parameters {
real zbeta0[Nsubj] ;
real zbeta1[Nsubj] ;
real zbeta2[Nsubj] ;
real<lower=0> zsigma ;
real zbeta0mu ;
real zbeta1mu ;
real zbeta2mu ;
real<lower=0> zbeta0sigma ;
real<lower=0> zbeta1sigma ;
real<lower=0> zbeta2sigma ;
real<lower=0> nuMinusOne ;
}
transformed parameters {
real<lower=0> nu ;
real beta0[Nsubj] ;
real beta1[Nsubj] ;
real beta2[Nsubj] ;
real<lower=0> sigma ;
real beta0mu ;
real beta1mu ;
real beta2mu ;
nu <- nuMinusOne+1 ;
// Transform to original scale:
for ( j in 1:Nsubj ) { // could be vectorized...?
beta2[j] <- zbeta2[j]*ysd/square(xsd) ;
beta1[j] <- zbeta1[j]*ysd/xsd - 2*zbeta2[j]*xm*ysd/square(xsd) ;
beta0[j] <- zbeta0[j]*ysd + ym - zbeta1[j]*xm*ysd/xsd
+ zbeta2[j]*square(xm)*ysd/square(xsd) ;
}
beta2mu <- zbeta2mu*ysd/square(xsd) ;
beta1mu <- zbeta1mu*ysd/xsd - 2*zbeta2mu*xm*ysd/square(xsd) ;
beta0mu <- zbeta0mu*ysd + ym - zbeta1mu*xm*ysd/xsd
+ zbeta2mu*square(xm)*ysd/square(xsd) ;
sigma <- zsigma * ysd ;
}
model {
zbeta0mu ˜ normal( 0 , 10 ) ;
zbeta1mu ˜ normal( 0 , 10 ) ;
zbeta2mu ˜ normal( 0 , 10 ) ;
zsigma ˜ uniform( 1.0E-3 , 1.0E+3 ) ;
zbeta0sigma ˜ uniform( 1.0E-3 , 1.0E+3 ) ;
zbeta1sigma ˜ uniform( 1.0E-3 , 1.0E+3 ) ;
zbeta2sigma ˜ uniform( 1.0E-3 , 1.0E+3 ) ;
nuMinusOne ˜ exponential(1/29.0) ;
zbeta0 ˜ normal( zbeta0mu , zbeta0sigma ) ; // vectorized
zbeta1 ˜ normal( zbeta1mu , zbeta1sigma ) ; // vectorized
zbeta2 ˜ normal( zbeta2mu , zbeta2sigma ) ; // vectorized
for ( i in 1:Ntotal ) {
zy[i] ˜ student_t(
nu ,
zbeta0[s[i]] + zbeta1[s[i]] * zx[i] + zbeta2[s[i]] * square(zx[i]) ,
zw[i]*zsigma ) ;
}
}
```
Review Section 14.4 (p. 414) for hints about programming Stan.
### A
Run Stan on the family-income data, so it achieves the same ESS as the JAGS
program for the group-level trend coefficients. How long (in real time) do Stan and
JAGS take? Does Stan more consistently converge than JAGS? Does Stan produce better
chains for the normality and noise parameters?


### B
Now repeat on the fictitious data of Figure 17.8, which typically gives JAGS and
Stan troubles of differing sorts as described Footnote 5 (p. 504). Try to produce examples
of these troubles and discuss. Which type of trouble is more tolerable, autocorrelation in
the normality parameter (in JAGS) or “bumps” in the regression coefficients (in Stan)?